<a href="https://colab.research.google.com/github/AlexandreBourrieau/ML/blob/main/Carnets%20Jupyter/S%C3%A9ries%20temporelles/Bitcoin/Bitcoin_Create_Features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chargement des données

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd
from sklearn import preprocessing

In [ ]:
!wget --no-check-certificate --content-disposition "https://raw.githubusercontent.com/AlexandreBourrieau/ML/main/Carnets%20Jupyter/S%C3%A9ries%20temporelles/Bitcoin/technical_indicators.py"
!wget --no-check-certificate --content-disposition "https://raw.githubusercontent.com/AlexandreBourrieau/ML/main/Carnets%20Jupyter/S%C3%A9ries%20temporelles/Bitcoin/All_Infos_Bitcoin.csv"


Charge la série sous Pandas et affiche les informations du fichier :

In [ ]:
# Création de la série sous Pandas
df = pd.read_csv("All_Infos_Bitcoin.csv")
df

In [ ]:
# Affiche les types
df.dtypes

In [ ]:
# Conversion des dates en type Datetime
df.Dates = pd.to_datetime(df.Dates)

# Mise en place de l'index au format DatetimeIndex
df = df.set_index('Dates')
df.dtypes

# Pré-traitement des données

**1. Affichage des données**

On demande à échantillonner les données sur 1 jour :

In [ ]:
# Echantillonnage de la série sur 1min
df = df.resample('1D').asfreq()
df

Normalise toutes les données pour les afficher :

In [ ]:
normalized_df=(df-df.mean())/df.std()

In [ ]:
date_debut = "2009-01-03"
date_fin = "2021-04-27"
f1, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10))
f1.subplots_adjust(hspace=0.3,wspace=0.2)

ax1.plot(normalized_df['Price'].loc[date_debut:date_fin])
ax2.plot(normalized_df.loc[date_debut:date_fin])

On sélectionne un intervalle de temps et on regarde si cet intervalle à l'air correct :

In [ ]:
date_debut = "2015-01-01"
date_fin = "2021-04-27"
f1, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10))
f1.subplots_adjust(hspace=0.3,wspace=0.2)

ax1.plot(normalized_df['Price'].loc[date_debut:date_fin])
ax2.plot(normalized_df.loc[date_debut:date_fin])

Création d'un nouveau dataframe sur l'intervalle choisi :

In [ ]:
df_etude = df.loc[date_debut:date_fin].copy()
df_etude

**2. Recherche des données manquantes**

On commence par rechercher combien de données sont manquantes dans chaque colonne sur l'intervalle choisi :

In [ ]:
for col in df_etude.columns:
  data_manquantes = sum(np.isnan(df_etude[col]))
  print ("Données manquantes (%s) : %s" %(col,data_manquantes))


On commence par tenter d'estimer les données manquantes à l'aide d'une interpolation linéaire à l'aide de la fonction [interpolate](https://pandas.pydata.org/docs/reference/api/pandas.Series.interpolate.html#pandas.Series.interpolate) de Pandas, puis on complète avec la méthode [fillna](https://pandas.pydata.org/docs/reference/api/pandas.Series.fillna.html) de Pandas avec la fonctionnalité de type `backfill` :

In [ ]:
df_etude = df_etude.interpolate(method="slinear")
df_etude = df_etude.fillna(method="backfill")

for col in df_etude.columns:
  data_manquantes = sum(np.isnan(df_etude[col]))
  print ("Données manquantes (%s) : %s" %(col,data_manquantes))

In [ ]:
df_etude['Google GTrends']

In [ ]:
df_etude = df_etude.fillna(method='ffill')
df_etude['Google GTrends']

In [ ]:
for col in df_etude.columns:
  data_manquantes = sum(np.isnan(df_etude[col]))
  print ("Données manquantes (%s) : %s" %(col,data_manquantes))

**3. Recherche des anomalies**

Les anomalies sont fréquentes dans les séries temporelles, et la performance des prédictions est souvent améliorée lorsque ces anomalies sont traitées.  
Pour avoir un apperçu de ces éventuelles anomalies, nous allons utiliser la méthode ["Isolation Forest"](https://scikit-learn.org/stable/modules/outlier_detection.html#isolation-forest) disponnible dans Scikit-learn.  

Les paramètres utilisés sont les suivants :
 - **n_estimators** : C'est le nombre de sous-groupes d'échantillons à utiliser. Une valeur de 128 ou 256 est préconnisée dans le document de recherche.
 - **max_samples** : C'est le nombre d'échantillons maximum à utiliser. Nous utiliserons l'ensemble des échantillons.
 - **max_features** :  C'est le nombre de motifs aléatoirement choisis sur chaque noeud de l'arbre. Nous choisirons un seul motif.
 - **contamination** : C'est le pourcentage estimé d'anomalies dans les données. Ce paramètre permet de régler la sensibilité de l'algorithme. On va commencer avec 5% et affiner si nécessaire par la suite.

Voici un exemple d'utilisation de l'algorithme pour détecter des anomalies sur les prix :

In [ ]:
# Initialise le modèle
from sklearn.ensemble import IsolationForest

clf = IsolationForest(n_estimators=256,max_samples=df_etude['Price'].size, contamination=0.01,max_features=1, verbose=1)
clf.fit(df_etude['Price'].values.reshape(-1,1))

In [ ]:
# Réalise les prédictions
pred = clf.predict(df_etude['Price'].values.reshape(-1,1))
pred

On ajoute maintenant ces informations dans la série journalière et on affiche les informations :

In [ ]:
# Ajoute une colonne "Anomalie" dans la série
df_etude['Anomalies']=pred
df_etude['Anomalies'] = df_etude['Anomalies'].apply(lambda x: 1 if (x==-1) else 0)

# Affiche les informations sur les anomalies
print(df_etude['Anomalies'].value_counts())

In [ ]:
# Affiche la série

fig = px.line(x=df_etude.index,y=df_etude['Price'],title="Evolution du prix du BTC")
fig.add_trace(px.scatter(x=df_etude.index,y=df_etude['Anomalies']*df_etude['Price'],color=df_etude['Anomalies'].astype(np.bool)).data[0])

fig.update_xaxes(rangeslider_visible=True)
yaxis=dict(autorange = True,fixedrange= False)
fig.update_yaxes(yaxis)
fig.show()

On fait les mêmes recherches sur les autres colonnes :

In [ ]:
num_colonne = 10

clf = IsolationForest(n_estimators=256,max_samples=df_etude[df_etude.columns[num_colonne]].size, contamination=0.01,max_features=1, verbose=1)
clf.fit(df_etude[df_etude.columns[num_colonne]].values.reshape(-1,1))
pred = clf.predict(df_etude[df_etude.columns[num_colonne]].values.reshape(-1,1))

# Ajoute une colonne "Anomalie" dans la série
df_etude['Anomalies']= pred
df_etude['Anomalies'] = df_etude['Anomalies'].apply(lambda x: 1 if (x==-1) else 0)

# Affiche la série
fig = px.line(x=df_etude.index,y=df_etude[df_etude.columns[num_colonne]],title=df_etude.columns[num_colonne])
fig.add_trace(px.scatter(x=df_etude.index,y=df_etude['Anomalies']*df_etude[df_etude.columns[num_colonne]],color=df_etude['Anomalies'].astype(np.bool)).data[0])

fig.update_xaxes(rangeslider_visible=True)
yaxis=dict(autorange = True,fixedrange= False)
fig.update_yaxes(yaxis)
fig.show()

On supprime maintenant la colonne "Anomalies" :

In [ ]:
df_etude = df_etude.drop(columns="Anomalies")

# Création des indicateurs techniques

In [ ]:
!pip install pandas_ta
import pandas_ta as ta

**1. SMA (Simple Moving Average)**

In [ ]:
n = [3,7,14,30,90]

df_indicateurs = pd.DataFrame(index=df_etude.index)

# SMA sur n jours
for i in n:
  for col in df_etude.columns:
    nom = col + "_sma" + str(i)
    df_indicateurs[nom] = ta.sma(df_etude[col],i)


In [ ]:
df_indicateurs.head()

In [ ]:
import plotly.graph_objects as go

col = "Price"
indicateur = "sma"

fig = go.Figure()

fig.add_trace(go.Scatter(x=df_etude.index,y=df_etude[col],line=dict(color='blue', width=1),name=col))

fig.add_trace(go.Scatter(x=df_indicateurs.index,y=df_indicateurs[col+'_'+indicateur+'3'],line=dict(color='green', width=1),name=indicateur+'3'))
fig.add_trace(go.Scatter(x=df_indicateurs.index,y=df_indicateurs[col+'_'+indicateur+'7'],line=dict(color='red', width=1),name=indicateur+'7'))
fig.add_trace(go.Scatter(x=df_indicateurs.index,y=df_indicateurs[col+'_'+indicateur+'14'],line=dict(color='black', width=1),name=indicateur+'14'))

fig.update_xaxes(rangeslider_visible=True)
yaxis=dict(autorange = True,fixedrange= False)
fig.update_yaxes(yaxis)
fig.show()


**2. EMA (Exponential Moving Average)**

In [ ]:
n = [3,7,14,30,90]

# EMA sur n jours
for i in n:
  for col in df_etude.columns:
    nom = col + "_ema" + str(i)
    df_indicateurs[nom] = ta.ema(df_etude[col],i)

In [ ]:
import plotly.graph_objects as go

col = "Price"
indicateur = "ema"

fig = go.Figure()

fig.add_trace(go.Scatter(x=df_etude.index,y=df_etude[col],line=dict(color='blue', width=1),name=col))

fig.add_trace(go.Scatter(x=df_indicateurs.index,y=df_indicateurs[col+'_'+indicateur+'3'],line=dict(color='green', width=1),name=indicateur+'3'))
fig.add_trace(go.Scatter(x=df_indicateurs.index,y=df_indicateurs[col+'_'+indicateur+'7'],line=dict(color='red', width=1),name=indicateur+'7'))
fig.add_trace(go.Scatter(x=df_indicateurs.index,y=df_indicateurs[col+'_'+indicateur+'14'],line=dict(color='black', width=1),name=indicateur+'14'))

fig.update_xaxes(rangeslider_visible=True)
yaxis=dict(autorange = True,fixedrange= False)
fig.update_yaxes(yaxis)
fig.show()


**3. RSI (Relative Strength Index)**

In [ ]:
n = [3,7,14,30,90]

# RSI sur n jours
for i in n:
  for col in df_etude.columns:
    nom = col + "_rsi" + str(i)
    df_indicateurs[nom] = ta.rsi(df_etude[col],i)

In [ ]:
import plotly.graph_objects as go

col = "Price"
indicateur = "rsi"

fig = go.Figure()

fig.add_trace(go.Scatter(x=df_etude.index,y=df_etude[col],line=dict(color='blue', width=1),name=col))

fig.add_trace(go.Scatter(x=df_indicateurs.index,y=df_indicateurs[col+'_'+indicateur+'3'],line=dict(color='green', width=1),name=indicateur+'3'))
fig.add_trace(go.Scatter(x=df_indicateurs.index,y=df_indicateurs[col+'_'+indicateur+'7'],line=dict(color='red', width=1),name=indicateur+'7'))
fig.add_trace(go.Scatter(x=df_indicateurs.index,y=df_indicateurs[col+'_'+indicateur+'14'],line=dict(color='black', width=1),name=indicateur+'14'))

fig.update_xaxes(rangeslider_visible=True)
yaxis=dict(autorange = True,fixedrange= False)
fig.update_yaxes(yaxis)
fig.show()

**4. WMA (Weighted Moving Average)**

In [ ]:
n = [3,7,14,30,90]

# WMA sur n jours
for i in n:
  for col in df_etude.columns:
    nom = col + "_wma" + str(i)
    df_indicateurs[nom] = ta.wma(df_etude[col],i)

In [ ]:
import plotly.graph_objects as go

col = "Price"
indicateur = "wma"

fig = go.Figure()

fig.add_trace(go.Scatter(x=df_etude.index,y=df_etude[col],line=dict(color='blue', width=1),name=col))

fig.add_trace(go.Scatter(x=df_indicateurs.index,y=df_indicateurs[col+'_'+indicateur+'3'],line=dict(color='green', width=1),name=indicateur+'3'))
fig.add_trace(go.Scatter(x=df_indicateurs.index,y=df_indicateurs[col+'_'+indicateur+'7'],line=dict(color='red', width=1),name=indicateur+'7'))
fig.add_trace(go.Scatter(x=df_indicateurs.index,y=df_indicateurs[col+'_'+indicateur+'14'],line=dict(color='black', width=1),name=indicateur+'14'))

fig.update_xaxes(rangeslider_visible=True)
yaxis=dict(autorange = True,fixedrange= False)
fig.update_yaxes(yaxis)
fig.show()

**5. STD (Standard Deviation)**

In [ ]:
n = [3,7,14,30,90]

# STD sur n jours
for i in n:
  for col in df_etude.columns:
    nom = col + "_std" + str(i)
    df_indicateurs[nom] = ta.stdev(df_etude[col],i)

In [ ]:
import plotly.graph_objects as go

col = "Price"
indicateur = "std"

fig = go.Figure()

fig.add_trace(go.Scatter(x=df_etude.index,y=df_etude[col],line=dict(color='blue', width=1),name=col))

fig.add_trace(go.Scatter(x=df_indicateurs.index,y=df_indicateurs[col+'_'+indicateur+'3'],line=dict(color='green', width=1),name=indicateur+'3'))
fig.add_trace(go.Scatter(x=df_indicateurs.index,y=df_indicateurs[col+'_'+indicateur+'7'],line=dict(color='red', width=1),name=indicateur+'7'))
fig.add_trace(go.Scatter(x=df_indicateurs.index,y=df_indicateurs[col+'_'+indicateur+'14'],line=dict(color='black', width=1),name=indicateur+'14'))

fig.update_xaxes(rangeslider_visible=True)
yaxis=dict(autorange = True,fixedrange= False)
fig.update_yaxes(yaxis)
fig.show()

**6. Skew (SKEW)**

In [ ]:
n = [3,7,14,30,90]

# SKEW sur n jours
for i in n:
  for col in df_etude.columns:
    nom = col + "_skew" + str(i)
    df_indicateurs[nom] = ta.skew(df_etude[col],i)

In [ ]:
import plotly.graph_objects as go

col = "Price"
indicateur = "skew"

fig = go.Figure()

fig.add_trace(go.Scatter(x=df_etude.index,y=df_etude[col],line=dict(color='blue', width=1),name=col))

fig.add_trace(go.Scatter(x=df_indicateurs.index,y=df_indicateurs[col+'_'+indicateur+'3'],line=dict(color='green', width=1),name=indicateur+'3'))
fig.add_trace(go.Scatter(x=df_indicateurs.index,y=df_indicateurs[col+'_'+indicateur+'7'],line=dict(color='red', width=1),name=indicateur+'7'))
fig.add_trace(go.Scatter(x=df_indicateurs.index,y=df_indicateurs[col+'_'+indicateur+'14'],line=dict(color='black', width=1),name=indicateur+'14'))

fig.update_xaxes(rangeslider_visible=True)
yaxis=dict(autorange = True,fixedrange= False)
fig.update_yaxes(yaxis)
fig.show()

**7. TRIX (Triple Exponential Moving Average)**

In [ ]:
n = [3,7,14,30,90]

# TRIX sur n jours
for i in n:
  for col in df_etude.columns:
    nom = col + "_trix" + str(i)
    df_indicateurs[nom] = ta.tema(df_etude[col],i)

In [ ]:
import plotly.graph_objects as go

col = "Price"
indicateur = "trix"

fig = go.Figure()

fig.add_trace(go.Scatter(x=df_etude.index,y=df_etude[col],line=dict(color='blue', width=1),name=col))

fig.add_trace(go.Scatter(x=df_indicateurs.index,y=df_indicateurs[col+'_'+indicateur+'3'],line=dict(color='green', width=1),name=indicateur+'3'))
fig.add_trace(go.Scatter(x=df_indicateurs.index,y=df_indicateurs[col+'_'+indicateur+'7'],line=dict(color='red', width=1),name=indicateur+'7'))
fig.add_trace(go.Scatter(x=df_indicateurs.index,y=df_indicateurs[col+'_'+indicateur+'14'],line=dict(color='black', width=1),name=indicateur+'14'))

fig.update_xaxes(rangeslider_visible=True)
yaxis=dict(autorange = True,fixedrange= False)
fig.update_yaxes(yaxis)
fig.show()

**8. ROC (Rate Of Change)**

In [ ]:
n = [3,7,14,30,90]

# TRIX sur n jours
for i in n:
  for col in df_etude.columns:
    nom = col + "_roc" + str(i)
    df_indicateurs[nom] = ta.roc(df_etude[col],i)

In [ ]:
import plotly.graph_objects as go

col = "Price"
indicateur = "roc"

fig = go.Figure()

fig.add_trace(go.Scatter(x=df_etude.index,y=df_etude[col],line=dict(color='blue', width=1),name=col))

fig.add_trace(go.Scatter(x=df_indicateurs.index,y=df_indicateurs[col+'_'+indicateur+'3'],line=dict(color='green', width=1),name=indicateur+'3'))
fig.add_trace(go.Scatter(x=df_indicateurs.index,y=df_indicateurs[col+'_'+indicateur+'7'],line=dict(color='red', width=1),name=indicateur+'7'))
fig.add_trace(go.Scatter(x=df_indicateurs.index,y=df_indicateurs[col+'_'+indicateur+'14'],line=dict(color='black', width=1),name=indicateur+'14'))

fig.update_xaxes(rangeslider_visible=True)
yaxis=dict(autorange = True,fixedrange= False)
fig.update_yaxes(yaxis)
fig.show()

In [ ]:
df_etude

# Fusion des données et sauvegarde des données

In [ ]:
df_complet = pd.merge(df_etude,df_indicateurs,how='outer',on='Dates')

In [ ]:
df_complet.to_csv("Bitcoin_complet.csv")